In [ ]:

import time
import chess
import chess.svg

piece_values = {
    chess.PAWN: 100,
    chess.KNIGHT: 300,
    chess.BISHOP: 300,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 0
}

piece_position = {
    chess.PAWN: [
        0,  0,  0,  0,  0,  0,  0,  0,
        50, 50, 50, 50, 50, 50, 50, 50,
        10, 10, 20, 30, 30, 20, 10, 10,
        5,  5, 10, 10, 10, 10,  5,  5,
        0,  0,  0, 200, 200,  0,  0,  0,
        5, -5,-10,  0,  0,-10, -5,  5,
        5, 10, 10,-20,-20, 10, 10,  5,
        0,  0,  0,  0,  0,  0,  0,  0
    ],

    chess.KNIGHT: [
        -50,-40,-30,-30,-30,-30,-40,-50,
        -40,-20,  0,  0,  0,  0,-20,-40,
        -30,  0, 10, 15, 15, 10,  0,-30,
        -30,  5, 15, 20, 20, 15,  5,-30,
        -30,  0, 15, 20, 20, 15,  0,-30,
        -30,  5, 10, 15, 15, 10,  5,-30,
        -40,-20,  0,  5,  5,  0,-20,-40,
        -50,-40,-30,-30,-30,-30,-40,-50,
    ],

    chess.BISHOP: [
        -20,-10,-10,-10,-10,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5, 10, 10,  5,  0,-10,
        -10,  5,  5, 10, 10,  5,  5,-10,
        -10,  0, 10, 10, 10, 10,  0,-10,
        -10, 10, 10, 10, 10, 10, 10,-10,
        -10,  5,  0,  0,  0,  0,  5,-10,
        -20,-10,-10,-10,-10,-10,-10,-20,
    ],

    chess.ROOK: [
        0,  0,  0,  0,  0,  0,  0,  0,
        5, 10, 10, 10, 10, 10, 10,  5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        0,  0,  0,  5,  5,  0,  0,  0
    ],

    chess.QUEEN: [
        -20,-10,-10, -5, -5,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5,  5,  5,  5,  0,-10,
        -5,  0,  5,  5,  5,  5,  0, -5,
        0,  0,  5,  5,  5,  5,  0, -5,
        -10,  5,  5,  5,  5,  5,  0,-10,
        -10,  0,  5,  0,  0,  0,  0,-10,
        -20,-10,-10, -5, -5,-10,-10,-20
    ],
    
    chess.KING: [
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -20,-30,-30,-40,-40,-30,-30,-20,
        -10,-20,-20,-20,-20,-20,-20,-10,
        20, 20,  0,  0,  0,  0, 20, 20,
        20, 30, 10,  0,  0, 10, 30, 20
    ]
}


def evaluate_board(position):
    score = 0
    for square in chess.SQUARES:
        piece = position.piece_at(square)
        if piece:
            piece_type = piece.piece_type
            if piece.color == chess.WHITE:
                mirrored = chess.square_mirror(square)
                score += piece_values[piece_type]
                score += piece_position[piece_type][mirrored] 
            else:
                score -= piece_values[piece_type]
                score -= piece_position[piece_type][square]
    return score/100

# push() / pop()
def minimax_pushpop(position, depth, alpha, beta, maximizing):
    if depth == 0 or position.is_game_over():
        return evaluate_board(position), None
    
    best_move = None
    if maximizing:
        max_eval = float('-inf')
        for move in position.legal_moves:
            position.push(move)
            eval, _ = minimax_pushpop(position, depth-1, alpha, beta, False)
            position.pop()
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for move in position.legal_moves:
            position.push(move)
            eval, _ = minimax_pushpop(position, depth-1, alpha, beta, True)
            position.pop()
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move


# copy()
def minimax_copy(position, depth, alpha, beta, maximizing):
    if depth == 0 or position.is_game_over():
        return evaluate_board(position), None
    
    best_move = None
    if maximizing:
        max_eval = float('-inf')
        for move in position.legal_moves:
            new_board = position.copy()
            new_board.push(move)
            eval, _ = minimax_copy(new_board, depth-1, alpha, beta, False)
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for move in position.legal_moves:
            new_board = position.copy()
            new_board.push(move)
            eval, _ = minimax_copy(new_board, depth-1, alpha, beta, True)
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move
    

board = chess.Board()
depth = 6 

start = time.time()
_, best_move_push = minimax_pushpop(board, depth, float('-inf'), float('inf'), True)
end = time.time()
print(f"push/pop czas: {end - start:.4f}s")

start = time.time()
_, best_move_copy = minimax_copy(board, depth, float('-inf'), float('inf'), True)
end = time.time()
print(f"copy()    czas: {end - start:.4f}s")

print(f"push/pop best move: {best_move_push}")
print(f"copy()    best move: {best_move_copy}")



push/pop czas: 93.7054s
copy()    czas: 137.5239s
push/pop best move: d2d4
copy()    best move: d2d4
